In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
tf = pd.read_csv("../input/train.csv")
ft = pd.read_csv("../input/test.csv")

In [ ]:
tf.head()

In [ ]:
ft.head()

In [ ]:
tf['spacegroup'].value_counts(normalize= True).plot.bar()
plt.title('SpaceGroup distribution in the training set')
plt.ylabel('% of crystals in spacegroup')
plt.xlabel('Space group')
plt.show()

In [ ]:
ft['spacegroup'].value_counts(normalize=True).plot.bar()

In [ ]:
tf.describe()

In [ ]:
ft.describe()



#### Okay, we're in luck. The same spacegroup distribution is roughly the same between test and training set, so our models are likely to be relevant. Now let's look at the individual space groups with some information from the Bilbao Crystallographic Server.

In [ ]:
tf.groupby(['spacegroup'])['percent_atom_in'].describe()

In [ ]:
tf.groupby(['spacegroup'])['percent_atom_al'].describe()

In [ ]:
tf.groupby(['spacegroup'])['percent_atom_ga'].describe()

In [ ]:
tf.groupby(['spacegroup'])['formation_energy_ev_natom'].describe()

In [ ]:
tf[tf['formation_energy_ev_natom'] < 0.05].groupby(['spacegroup'])['formation_energy_ev_natom'].describe()

In [ ]:
tf.groupby(['spacegroup'])['bandgap_energy_ev'].describe()

In [ ]:
tf[tf['bandgap_energy_ev'] >= 3.2].groupby(['spacegroup'])['bandgap_energy_ev'].describe()

In [ ]:
tf[(tf['bandgap_energy_ev'] >= 3.2) & (tf['formation_energy_ev_natom'] < 0.05)].groupby(['spacegroup'])['bandgap_energy_ev'].describe()

In [ ]:
tf.describe()

In [ ]:
id = tf['id']
tf.drop('id', axis=1, inplace=True)

In [ ]:
tf.head()

In [ ]:
#Performing feature scaling
y = tf['formation_energy_ev_natom']
y1 = tf['bandgap_energy_ev']
tf.drop('formation_energy_ev_natom', axis = 1, inplace= True)
tf.drop('bandgap_energy_ev', axis = 1, inplace=True)
tf.head()

In [ ]:
X = np.array(tf)
X.shape
y = np.array(y.tolist())
y.shape

In [ ]:
#Feature scaling usig min -max scaling
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state =0)
X1_train,X1_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.6, random_state =0)
sh = ShuffleSplit(n_splits=4, test_size=0.4, random_state=0)
sh.get_n_splits(X, y)
print(X_train.shape, y_train.shape)
id1 = ft['id']
print(ft.head())
ft.drop('id', axis=1, inplace =True)
T1 = np.array(ft)
print(T1.shape)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
clf_3 = SVR(kernel = 'rbf', degree=6, gamma='auto', max_iter=-1,C=0.5, epsilon=0)
print(clf_3.fit(X_train, y_train))
clf_3_1 = SVR(kernel = 'rbf', degree=6, gamma='auto', max_iter=-1,C=0.5, epsilon=0)
print("the fitting function is for energybandgap")
print(clf_3_1.fit(X1_train, y1_train))
print("##########################################")
clf_2 = RandomForestRegressor(n_estimators= 11, max_features='auto',max_depth=10,random_state=0 )
print(clf_2.fit(X_train, y_train))
print("##########################################")
clf_2_1 = RandomForestRegressor(n_estimators= 11, max_features='auto',max_depth=10,random_state=0 )
print("the fitting function with Random forest is for energybandgap")
print(clf_2_1.fit(X1_train, y1_train))

In [ ]:
print(clf_3.score(X_test, y_test)*100)
print("This the accuracy for energy bandgap",clf_3_1.score(X1_test, y1_test)*100)

In [ ]:
print(clf_2.score(X_test, y_test)*100)
print("This the accuracy for energy bandgap",clf_2_1.score(X1_test, y1_test)*100)

In [ ]:
a = clf_2.predict(T1)
b = clf_2_1.predict(T1)
print(a)

In [ ]:
print(b)

In [ ]:
final = pd.DataFrame()
final['id']=id1
final['formation_energy_ev_natom'] = a
final['bandgap_energy_ev'] = b
final.info()

In [ ]:
final.to_csv("final.csv", index=False)